In [3]:
# !pip install camel-tools==1.2.0 farasapy nltk Arabic-Stopwords scikit-learn gensim pandas
# !camel_data light
# import nltk
# nltk.download('stopwords')

In [16]:
# built in modules
import numpy as np
import pandas as pd
# from sklearn import metrics 

# .py files 
from preprocess import Preprocess
from feature_extraction import *

from classical_models import *
# Can use this library to reload a specific module if the notebook can't see changes in the imported module
import importlib

Stance detection labels meaning is as follows:
1. Positive (1): means that the tweet author encourages and supports vaccination.
2. Negative (-1): means that the tweet author refuses vaccination.
3. Neutral (0): means that the tweet neither supports nor refuses vaccination.

Category labels meaning is as follows:
1. Info_News: Information about vaccination.
2. Celebrities: mentioning celebrities taking vaccinations.
3. Plan: Governmental plan or progress of vaccination.
4. Request: Requests from governments regarding the vaccination process.
5. Rumor: the tweet is a rumor.
6. Advice: Advice related to the virus or the vaccination
7. Restriction: Restrictions due to the virus e.g. traveling.
8. Personal: Personal opinion or story about vaccination.
9. Unrelated: Unrelated to vaccination.
10.Others: Vaccination related but not one of the above.

In [5]:
# !unzip Dataset.zip
t = pd.read_csv('./Dataset/train.csv')
d = pd.read_csv('./Dataset/dev.csv')
# ts = pd.read_csv('./Dataset/test.csv')
#
# ov = pd.read_csv('/content/Dataset/classification_train_sample1.csv')
# ov2 = pd.read_csv('/content/Dataset/cat_somesample.csv')

In [6]:
categories = ["info_news", "celebrity", "plan", "requests", "rumors", "advice", "restrictions", "personal", "unrelated", "others"]
def encode_category(y):
    '''
    Input: y a list of string labels for the category of each document 
    Output: a list of encoded 10 sized array for the category of each doc 
            for "other" category , it has an array =[0 0 0 0 0 0 0 0 0 1] 
    '''
    return [categories.index(ele) for ele in y]

### Understanding the Data
- 80% tweets are positive (class 1), the rest are neutral and negative.
- 50% tweets belongs to info_news category.

In [7]:
# # analyze dataset
# print("counts for each lablel :")
# print(t['category'].value_counts(normalize=True))
# print(ov['category'].value_counts(normalize=True))
# print(ov2['category'].value_counts(normalize=True))
# print(d['category'].value_counts(normalize=True))

In [8]:
# print("####################################")
# print("duplicated rows :")
# print(t[t.duplicated()])

In [9]:
X_train = t['text']
# X_trainov = ov['text']
# X_trainov2 = ov2['text']
X_dev = d['text']
# X_test = ts['text']
#
Y_train = t[['stance', 'category']]
# Y_trainov = ov['category']
# Y_trainov2 = ov2['category']
Y_dev = d[['stance', 'category']]
# Y_test = ts['text']

#### Preprocessing

In [10]:
preprocess = Preprocess()
X_train = X_train.apply(preprocess.do_all)
# X_trainov = X_trainov.apply(preprocess.do_all)
# X_trainov2 = X_trainov2.apply(preprocess.do_all)
X_dev = X_dev.apply(preprocess.do_all)
# X_test = X_test.apply(preprocess.do_all)

Bert MODE False
Emojis:  raw


In [11]:
# r = random.randint(0, len(X_test))
# # r = 900
# print(r, '\n')
# print(ts.text[r], '\n')
# print(X_test[r], '\n')

In [12]:
# Print Vocabulary size
lst = [word for x in X_train for word in x]
vocab = set(lst)
print(len(lst))
print(len(vocab))
# print(vocab)

471829
31411


### Getting the train , test and dev features

In [13]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer =TfidfVectorizer(analyzer=lambda x: x)
# train_features= vectorizer.fit_transform(X)
# # return train_features.toarray(), test_features.toarray()

# # print 10 words with highest tfidf values for each CATEGORY
# feature_names = vectorizer.get_feature_names_out()
# for i, category in enumerate(categories):
#     lst = [0] *len(feature_names)
#     for row in train_features[np.array(yc) == i].toarray():
#         lst = [x + y for x, y in zip(lst, row)]
#     # top 10 words
#     top20 = np.argsort(lst)[::-1][:20]
#     # print names and values
#     print(category)
#     print([(feature_names[j], lst[j]) for j in top20])
#     print()

## Classical model Training 

- Classification Task Results:

|  | SVM | Random Forest | KNN |  Voting system |
| --------------- | --------------- | --------------- | --------------- | --------------- |
| TFIDF |69%  |  |  ||
| BOW | 54%  |   |   | |


- Stance Task Results:

|  | SVM | Random Forest | KNN |  Voting system |
| --------------- | --------------- | --------------- | --------------- | --------------- |
| TFIDF |81%  |  |  ||
| BOW | 80%  |   |   | |

In [14]:
def get_TFIDF(tr,ts):
    trw, tsw = TFIDF(tr, ts, 'word', 1, 2)
    trc, tsc = TFIDF(tr, ts, 'char', 2, 3)
    return np.concatenate((trw, trc), axis=1), np.concatenate((tsw, tsc), axis=1)

In [24]:
# train_features, test_features = get_TFIDF(X_train, X_dev)
train_features, test_features = SG(X_train, X_dev)

from sklearn import metrics

clss = [NBmodel, KNNmodel, LSVMmodel, LRmodel, RGmodel, RFmodel, DTmodel]
# clss = [SVMmodel]
# clss = [KNNmodel]
#
for cls in clss:
    #
    print(cls.__name__)
    # yc_pred = cls(Xtrain=train_features, y_train=Y_train['category'], X_test=test_features)
    ys_pred = cls(Xtrain=train_features, y_train=Y_train['stance'], X_test=test_features)
    #
    # print('========= Category =========')
    # print(metrics.classification_report(y_true=Y_dev['category'],y_pred=yc_pred))
    print('========= Stance =========')
    print(metrics.classification_report(y_true=Y_dev['stance'],y_pred=ys_pred))

# classify_accuracy=metrics.accuracy_score(y_true=yc_dev,y_pred=yc_pred)
# stance_accuracy=metrics.accuracy_score(y_true=ys_dev,y_pred=ys_pred)

NBmodel
========= Stance =========
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        70
           0       0.23      0.72      0.35       126
           1       0.89      0.67      0.76       804

    accuracy                           0.63      1000
   macro avg       0.37      0.46      0.37      1000
weighted avg       0.74      0.63      0.66      1000

KNNmodel


c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


========= Stance =========
              precision    recall  f1-score   support

          -1       0.05      0.01      0.02        70
           0       0.20      0.25      0.22       126
           1       0.81      0.84      0.83       804

    accuracy                           0.71      1000
   macro avg       0.36      0.37      0.36      1000
weighted avg       0.68      0.71      0.69      1000

LSVMmodel


c:\Program Files\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


========= Stance =========
              precision    recall  f1-score   support

          -1       0.17      0.01      0.03        70
           0       0.50      0.02      0.03       126
           1       0.81      1.00      0.89       804

    accuracy                           0.80      1000
   macro avg       0.49      0.34      0.32      1000
weighted avg       0.73      0.80      0.72      1000

LRmodel
========= Stance =========
              precision    recall  f1-score   support

          -1       0.08      1.00      0.14        70
           0       0.00      0.00      0.00       126
           1       1.00      0.12      0.21       804

    accuracy                           0.16      1000
   macro avg       0.36      0.37      0.12      1000
weighted avg       0.81      0.16      0.18      1000

RGmodel
========= Stance =========
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        70
           0       0.44      0.2

c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


========= Stance =========
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        70
           0       0.00      0.00      0.00       126
           1       0.80      1.00      0.89       804

    accuracy                           0.80      1000
   macro avg       0.27      0.33      0.30      1000
weighted avg       0.65      0.80      0.72      1000

DTmodel


c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


========= Stance =========
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        70
           0       0.11      0.02      0.03       126
           1       0.79      0.89      0.83       804

    accuracy                           0.71      1000
   macro avg       0.30      0.30      0.29      1000
weighted avg       0.65      0.71      0.67      1000



#### Prediction of Test Data

In [ ]:
# get output of test set
X_train_dev = np.concatenate((X_train, X_dev), axis=0)
Y_train_dev = np.concatenate((Y_train, Y_dev), axis=0)
print(X_train.shape, X_dev.shape, X_train_dev.shape)
print(Y_train_dev.shape)
train_features, test_features = TFIDF(X_train_dev, X_test)

yc_pred = LSVMmodel(Xtrain=train_features, y_train=Y_train_dev, X_test=test_features)

In [ ]:
# yc_pred
pred = [(i, vi) for i, vi in enumerate(yc_pred)]

print(pred[:15])
# print(pred.value_counts())
# print(metrics.classification_report(labels, pred))

#### Saving model to disk

In [ ]:
# from sklearn import svm
# clf=svm.LinearSVC(class_weight='balanced')
# clf.fit(X=train_features,y=Y_train_dev)

# from joblib import dump, load
# dump(clf, 'filename.joblib') 

#### Writing output to CSV

In [ ]:
# import csv
# with open('./out/test_cat.csv', "w",encoding="utf-8", newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(('id','stance'))
#     for row in pred:
#         writer.writerow(row)

# dd = pd.read_csv('./out/test_cat.csv')
# print(dd.head())

# print(dd.stance.value_counts())

In [ ]:
asds